In [ ]:
import pandas as pd
import numpy as np
import random
import warnings
warnings.filterwarnings('ignore')

In [ ]:
ibm=pd.read_csv(r"path\IBM.csv")
ibm_1=ibm[['Date', 'Open']]
ibm_2=ibm_1.iloc[23:-2,:]
ibm_2.set_index('Date', inplace=True)
fama_french=pd.read_csv(r"path\F-F.csv", skiprows=3)
fama_french.set_index('Date', inplace=True)
fama_french=fama_french.drop(['RF'], axis=1)

In [ ]:
ibm_2['Log_return']=np.log(ibm_2['Open'])-np.log(ibm_2['Open'].shift(+1))
ibm_2['Sign_return']=np.where(ibm_2.Log_return>=0,1,0)

In [ ]:
ff=fama_french.as_matrix()
y=ibm_2.loc[:,'Sign_return']

In [ ]:
#test set
len_test=68
y_train=y[:len(y)-len_test]
y_test=y[len(y)-len_test:]

In [ ]:
X_nn=[]
list_temp=[]
j=0
while j<len(ff)-4:

    d=0
    list_temp_1=[]
    list_temp_2=[]
    list_temp_3=[]
    list_temp_4=[]
    list_temp_5=[]
    while d<5:
        list_temp_1.append(ff[j+d][0])
        list_temp_2.append(ff[j+d][1])
        list_temp_3.append(ff[j+d][2])
        list_temp_4.append(ff[j+d][3])
        list_temp_5.append(ff[j+d][4])
        d+=1
    list_temp.append(list_temp_1)
    list_temp.append(list_temp_2)
    list_temp.append(list_temp_3)
    list_temp.append(list_temp_4)
    list_temp.append(list_temp_5)
    X_nn.append(list_temp)
    list_temp=[]
    j+=1

y_train=np.asarray(y_train).reshape((len(y_train), 1))
y_test=np.asarray(y_test).reshape((len(y_test), 1))
X_train=np.asarray(X_nn[:len(X_nn)-len_test])
X_test=np.asarray(X_nn[len(X_nn)-len_test:])

#Goal : move to 5 times 5 to lines of 25
p=0
list_temp=[]
new_format=[]
for i in X_train:
    new_format.append(list_temp)
    list_temp=[]
    for j in i:
        for k in j:
            list_temp.append(k)

new_format.append(list_temp)
new_format.pop(0)
X_train=np.asarray(new_format)

In [ ]:
#Goal : move to 5 times 5 to lines of 25
p=0
list_temp=[]
new_format=[]
for i in X_test:
    new_format.append(list_temp)
    list_temp=[]
    for j in i:
        for k in j:
            list_temp.append(k)

new_format.append(list_temp)
new_format.pop(0)
X_test=np.asarray(new_format)


In [ ]:
def sigmoid(x):
    return 1.0/(1+ np.exp(-x))

def sigmoid_derivative(x):
    return x * (1.0 - x)
def ReLU(x): 
    j=0
    for i in x:
        p=0
        for k in i:
            if k <0:
                x[j][p]=0
            p+=1
        j+=1
    return x

class NeuralNetwork:
    def __init__(self, x, y, lamb1, learning_r):
        self.input = x
        list_weights1 = []
        j=0
        while j<25:
            list_temp=[]
            k=j//5
            m=0
            list_temp+=[0]*10
            r=2*(k+1)
            b=r-2
            for i in range(b,r):
                list_temp[i]=random.uniform(-5,5)
            list_weights1.append(np.asarray(list_temp))
            j+=1
            
        self.weights1   = np.asarray(list_weights1)
        list_weights2=[]
        j=0
        while j<10:
            list_temp=[]
            k=j//2
            m=0
            list_temp+=[0]*5
            r=1*(k+1)
            b=r-1
            for i in range(b,r):
                list_temp[i]=random.uniform(-15,15)
            list_weights2.append(np.asarray(list_temp))
            j+=1
        
        self.weights2   = np.asarray(list_weights2)

        self.weights3   = np.random.uniform(low=-10,high=10, size=(5,1))
        self.y          = y
        self.output     = np.zeros(self.y.shape)
        
        self.lamb1=lamb1
        self.learning_r=learning_r
        
    def feedforward(self):
        self.layer1 = sigmoid(np.dot(self.input, self.weights1))
        self.layer2 = sigmoid(np.dot(self.layer1, self.weights2))
        self.output = sigmoid(np.dot(self.layer2, self.weights3))
        
    def backprop(self):
        # application of the chain rule to find derivative of the loss function with respect to weights
        self.list3=[]
        self.list2=[]
        self.list1=[]
        self.list_input=[]
        self.list_weights3=[]
        self.list_weights2=[]
        self.list_weights1=[]
        p=0
        while p<5:
            list2=[]
            for i in self.layer2:
                list2.append(i[p])
            list1=[]
            for i in self.layer1:
                list1.append(i[p*2:(p+1)*2])
            list_input=[]
            for i in self.input:
                list_input.append(i[p*5:(p+1)*5])
            
            list_weights2=[]
            f=0
            for i in self.weights2:
                if f>=(p)*2 and f<(p+1)*2:
                    list_weights2.append(i[p])
                f+=1

            list_weights1=[]

            for i in self.weights1[p*5:(p+1)*5]:
                list_weights1.append(i[p*2:(p+1)*2])

            
            self.list_input.append(np.asarray(list_input))
            self.list1.append(np.asarray(list1))
            self.list2.append(np.asarray(list2))
            self.list_weights2.append(np.asarray(list_weights2))
            self.list_weights1.append(np.asarray(list_weights1))
            p+=1
        
        self.list_input=np.asarray(self.list_input)     
        self.list1=np.asarray(self.list1)
        self.list2=np.asarray(self.list2)
        self.list2=self.list2.reshape(5,608,1)
        self.list_weights2=np.asarray(self.list_weights2)
        self.list_weights1=np.asarray(self.list_weights1)
        
        lambda_1=self.lamb1
        learning_rate=self.learning_r
        error_output=(self.y*np.log(self.output)+(1-self.y)*np.log(1-self.output))
        d_error_output=self.y/self.output - (1-self.y)/(1-self.output)

        d_weights3 = np.dot(self.layer2.T, (-2*(self.y-self.output) * sigmoid_derivative(self.output)))
        error_layer2= np.dot(-2*(self.y-self.output) * sigmoid_derivative(self.output), self.weights3.T)
        error_layer1=(np.dot((np.dot(-2*(self.y-self.output) * sigmoid_derivative(self.output), self.weights3.T) * sigmoid_derivative(self.layer2)),self.weights2.T))
        
        x=0
        error_neuron=[]
        while x<5:
            list_temp=[]
            for i in error_layer2:
                list_temp.append(i[x])
            error_neuron.append(np.asarray(list_temp))
            x+=1
            
        error_neuron=np.asarray(error_neuron) 
        f=0
        d_weights2=[]
        d_weights1=[]
        while f<5:
            d_weights2.append(np.dot(self.list1[f].T,(error_neuron[f].reshape(608,1) * sigmoid_derivative(self.list2[f]))))
            d_weights1.append(np.dot(self.list_input[f].T,(np.dot((error_neuron[f].reshape(608,1) * sigmoid_derivative(self.list2[f])),self.list_weights2[f].reshape(2,1).T)*sigmoid_derivative(self.list1[f]))))
            f+=1
        # update the weights with the derivative of the loss function

        self.weights3 -= learning_rate*d_weights3
        self.weights3 -= learning_rate*lambda_1*self.weights3

        s=0
        for d in d_weights2:
            for b in d:

                self.weights2[s][s//2]-=learning_rate*b
                s+=1
    
        
        e=0
        for i in d_weights1:
            for j in i:
                m=0
                for k in j:
                    self.weights1[e//2][(e//(5*2))*2+m]-=learning_rate*k
                    e+=1
                    m+=1
        
        self.weights2 -=learning_rate*lambda_1*self.weights2
        self.weights1 -=learning_rate*lambda_1*self.weights1
        

    def forecast (self, X_test, Y_test):
        test_layer1 = sigmoid(np.dot(X_test, self.weights1))
        test_layer2 = sigmoid(np.dot(test_layer1, self.weights2))
        self.test_output = sigmoid(np.dot(test_layer2, self.weights3))
        k=0
        hg=0
        bd=0
        hd=0
        bg=0
        liste_matrix=[]
        for i in self.test_output:
            if (i>0.5 and Y_test[k]==1):
                bd +=1
            elif (i<0.5 and Y_test[k]==0):
                hg +=1
            elif(i<0.5 and Y_test[k]==1):
                bg+=1
            else:
                hd +=1
            k+=1
        liste_matrix.append(hg)
        liste_matrix.append(bg)
        liste_matrix.append(hd)
        liste_matrix.append(bd)
        return liste_matrix
        
if __name__ == "__main__":
#     nn = NeuralNetwork(X_train,y_train, 0.33, 0.2)
#     for i in range(1000):
#         nn.feedforward()
#         nn.backprop()
#     test=nn.forecast(X_test, y_test)
#     print(test)
        
    maxi_out=0
    liste_max=[]
    liste_matrice=[]
    for p in np.arange(0.10,0.9,0.05):

        for j in np.arange(0,1,0.05):
            nn = NeuralNetwork(X_train,y_train, p, j)
            for i in range(400):
                nn.feedforward()
                nn.backprop()

            test=nn.forecast(X_test, y_test)
            if (test[0]+test[3])/(test[0]+test[1]+test[2]+test[3])>maxi_out:
                maxi_out=(test[0]+test[3])/(test[0]+test[1]+test[2]+test[3])
                liste_max.append((p,j))
                liste_matrice.append(test)
                print(test)

    print(maxi_out)
    print(liste_max)





In [ ]:
def sigmoid(x):
    return 1.0/(1+ np.exp(-x))

def sigmoid_derivative(x):
    return x * (1.0 - x)
def ReLU(x): 
    j=0
    for i in x:
        p=0
        for k in i:
            if k <0:
                x[j][p]=0
            p+=1
        j+=1
    return x
class NeuralNetwork:
    def __init__(self, x, y, lamb1, learning_r):
        self.input = x
        list_weights1 = []
        j=0
        while j<25:
            list_temp=[]
            k=j//5
            m=0
            list_temp+=[0]*10
            r=2*(k+1)
            b=r-2
            for i in range(b,r):
                list_temp[i]=random.uniform(-5,5)
            list_weights1.append(np.asarray(list_temp))
            j+=1
            
        self.weights1   = np.asarray(list_weights1)
        list_weights2=[]
        j=0
        while j<10:
            list_temp=[]
            k=j//2
            m=0
            list_temp+=[0]*5
            r=1*(k+1)
            b=r-1
            for i in range(b,r):
                list_temp[i]=random.uniform(-15,15)
            list_weights2.append(np.asarray(list_temp))
            j+=1
        
        self.weights2   = np.asarray(list_weights2)

        self.weights3   = np.random.uniform(low=-10,high=10, size=(5,1))
        self.y          = y
        self.output     = np.zeros(self.y.shape)
        
        self.lamb1=lamb1
        self.learning_r=learning_r
        
    def feedforward(self):
        self.layer1 = sigmoid(np.dot(self.input, self.weights1))
        self.layer2 = sigmoid(np.dot(self.layer1, self.weights2))
        self.output = sigmoid(np.dot(self.layer2, self.weights3))
        
    def backprop(self):
        # application of the chain rule to find derivative of the loss function with respect to weights
        self.list3=[]
        self.list2=[]
        self.list1=[]
        self.list_input=[]
        self.list_weights3=[]
        self.list_weights2=[]
        self.list_weights1=[]
        p=0
        while p<5:
            list2=[]
            for i in self.layer2:
                list2.append(i[p])
            list1=[]
            for i in self.layer1:
                list1.append(i[p*2:(p+1)*2])
            list_input=[]
            for i in self.input:
                list_input.append(i[p*5:(p+1)*5])
            
            list_weights2=[]
            f=0
            for i in self.weights2:
                if f>=(p)*2 and f<(p+1)*2:
                    list_weights2.append(i[p])
                f+=1

            list_weights1=[]

            for i in self.weights1[p*5:(p+1)*5]:
                list_weights1.append(i[p*2:(p+1)*2])

            
            self.list_input.append(np.asarray(list_input))
            self.list1.append(np.asarray(list1))
            self.list2.append(np.asarray(list2))
            self.list_weights2.append(np.asarray(list_weights2))
            self.list_weights1.append(np.asarray(list_weights1))
            p+=1
        
        self.list_input=np.asarray(self.list_input)     
        self.list1=np.asarray(self.list1)
        self.list2=np.asarray(self.list2)
        self.list2=self.list2.reshape(5,608,1)
        self.list_weights2=np.asarray(self.list_weights2)
        self.list_weights1=np.asarray(self.list_weights1)
        
        lambda_1=self.lamb1
        learning_rate=self.learning_r
        error_output=(self.y*np.log(self.output)+(1-self.y)*np.log(1-self.output))
        d_error_output=self.y/self.output - (1-self.y)/(1-self.output)

        d_weights3 = np.dot(self.layer2.T, (-2*(self.y-self.output) * sigmoid_derivative(self.output)))
        error_layer2= np.dot(-2*(self.y-self.output) * sigmoid_derivative(self.output), self.weights3.T)
        error_layer1=(np.dot((np.dot(-2*(self.y-self.output) * sigmoid_derivative(self.output), self.weights3.T) * sigmoid_derivative(self.layer2)),self.weights2.T))
        
        x=0
        error_neuron=[]
        while x<5:
            list_temp=[]
            for i in error_layer2:
                list_temp.append(i[x])
            error_neuron.append(np.asarray(list_temp))
            x+=1
            
        error_neuron=np.asarray(error_neuron) 
        f=0
        d_weights2=[]
        d_weights1=[]
        while f<5:
            d_weights2.append(np.dot(self.list1[f].T,(error_neuron[f].reshape(608,1) * sigmoid_derivative(self.list2[f]))))
            d_weights1.append(np.dot(self.list_input[f].T,(np.dot((error_neuron[f].reshape(608,1) * sigmoid_derivative(self.list2[f])),self.list_weights2[f].reshape(2,1).T)*sigmoid_derivative(self.list1[f]))))
            f+=1
        # update the weights with the derivative of the loss function

        self.weights3 -= learning_rate*d_weights3
        self.weights3 -= learning_rate*lambda_1*self.weights3

        s=0
        for d in d_weights2:
            for b in d:

                self.weights2[s][s//2]-=learning_rate*b
                s+=1
    
        
        e=0
        for i in d_weights1:
            for j in i:
                m=0
                for k in j:
                    self.weights1[e//2][(e//(5*2))*2+m]-=learning_rate*k
                    e+=1
                    m+=1
        
        self.weights2 -=learning_rate*lambda_1*self.weights2
        self.weights1 -=learning_rate*lambda_1*self.weights1
        

    def forecast (self, X_test, Y_test):
        test_layer1 = sigmoid(np.dot(X_test, self.weights1))
        test_layer2 = sigmoid(np.dot(test_layer1, self.weights2))
        self.test_output = sigmoid(np.dot(test_layer2, self.weights3))
        k=0
        hg=0
        bd=0
        hd=0
        bg=0
        liste_matrix=[]
        for i in self.test_output:
            if (i>0.5 and Y_test[k]==1):
                bd +=1
            elif (i<0.5 and Y_test[k]==0):
                hg +=1
            elif(i<0.5 and Y_test[k]==1):
                bg+=1
            else:
                hd +=1
            k+=1
        liste_matrix.append(hg)
        liste_matrix.append(bg)
        liste_matrix.append(hd)
        liste_matrix.append(bd)
        return liste_matrix
        
if __name__ == "__main__":
    nn = NeuralNetwork(X_train,y_train, 0.15, 0.3)
    l=0
    list_average_1=[]
    list_average_2=[]
    list_average_3=[]
    list_average_4=[]
    list_average_5=[]
    while l<100:
        for i in range(400):
            nn.feedforward()
            nn.backprop()
        test=nn.forecast(X_test, y_test)
        list_mat1=[]
        list_mat2=[]
        list_mat3=[]
        list_mat4=[]
        list_mat5=[]
        for j in nn.layer2:
            list_mat1.append(j[0])
            list_mat2.append(j[1])
            list_mat3.append(j[2])
            list_mat4.append(j[3])
            list_mat5.append(j[4])
        array_1=np.asarray(list_mat1)
        array_2=np.asarray(list_mat2)
        array_3=np.asarray(list_mat3)
        array_4=np.asarray(list_mat4)
        array_5=np.asarray(list_mat5)
        
        list_average_1.append((array_1*nn.weights3[0]).mean())
        list_average_2.append((array_2*nn.weights3[1]).mean())
        list_average_3.append((array_3*nn.weights3[2]).mean())
        list_average_4.append((array_4*nn.weights3[3]).mean())
        list_average_5.append((array_5*nn.weights3[4]).mean())
        
        l+=1

    

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns_plot=sns.distplot(np.asarray(list_average_1), hist=True, kde=True, color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4}).set_title("Distribution of the MKT-Rf estimated coefficients");
fig = sns_plot.get_figure()
fig.savefig("output_Mkt.png")